In [ ]:
# lets now self create the model and see what actually works 
'''
Preference / Reward model 
Here it learns to reward the 

** Dataset ** 

Prompt : Why is the color of sky blue ?
Chosen : Its because of the scattering of blue wavelength by air molecules 
Rejected : Because sky likes blue color


The reward model, is a linear layer at top, that learn to output whether this response is good or not 

Input1 to model : {Prompt + Chosen} ,  Output1 : 1  
Input2 to model : {Prompt + Rejected}  , Output2: 0 


Here the model learns to pick up a good response !

This is the logic that happens in PPO loop ! So there is nothing as seperate training for a PPO model directly just use this as a Value head  
'''

# Single runnable cell — minimal, targeted fixes only (copy-paste)
import os
import gc
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
    PreTrainedModel,
    GenerationConfig
)
from transformers.modeling_outputs import SequenceClassifierOutput
from trl import AutoModelForCausalLMWithValueHead
from transformers.modeling_outputs import ModelOutput
import torch.nn.functional as F

from dotenv import load_dotenv
load_dotenv()

# ------------------ USER-CHOICE (preserved) ------------------
BASE_MODEL = "HuggingFaceTB/SmolLM2-135M-Instruct"
DATASET_LINK = "Dahoas/rm-static"
DEVICE = 'mps'


import os 
FILE_PATH = os.path.dirname(os.path.abspath(""))
print(FILE_PATH)




In [ ]:

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL , cache_dir = FILE_PATH)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH)

dataset = load_dataset(DATASET_LINK, split="train[:20]", cache_dir=FILE_PATH)


In [ ]:
del model , tokenizer , dataset 

gc.collect()

In [ ]:
tokenizer.bos_token

In [ ]:
def prepare_dataset(example):
    prompt = tokenizer.bos_token + example['prompt']
    example['prompt'] = prompt
    tokenized_example =  tokenizer(prompt)
    input_ids = tokenized_example.input_ids
    attention_mask = tokenized_example.attention_mask
    mask = torch.tensor([1] * len(input_ids))

    return {
        'input_ids':input_ids, 
        'resp_mask': mask, 
        'attention_mask':mask
    }

dataset = dataset.map(prepare_dataset)

print(dataset[0])

In [ ]:
dataset[0]

In [ ]:
# import torch.nn.functional as F

# logp = None

# def next_token_prediction_values(logp:torch.Tensor, idx :int ) -> torch.Tensor:
#     return logp[:, idx, :] # output will be torch tensor 

# # resp_mask this is the response mask that means it tells which parts of the prompt to take in the input and which parts not to take !!
# def seq_logprob(model , input_ids , attention_mask, resp_mask):
#     '''
#     This is the log prob, this is the 
#     '''
    
#     if type(input_ids) is not torch.Tensor:
#         print(input_ids)
#         print(type(input_ids))
#         input_ids = torch.tensor(data = input_ids, dtype = torch.int32)
#         attention_mask = torch.tensor(data = attention_mask, dtype=torch.int32)

#     out = model(input_ids = input_ids.unsqueeze(0), attention_mask = attention_mask.unsqueeze(0))
#     print("out logits shape is ", out.logits.shape) # outputs the probability for all the input tokens , the next one will only be from the last one

#     # In the DPO, I am already passing the input to the model / the output also in the prompt itself ..  
#     logp = F.log_softmax(out.logits, dim = -1)
#     print("full logp shape is : ", logp.shape) # log outputs for all the next tokens  

#     # Now I need to find the last one, so we are taking the probability of the whole sequence .. the input + output ( prompt + response) 
#     # global logp
#     logp = F.log_softmax(out.logits, dim = -1)
#     print('logp shape is : ' , logp.shape) # 1 x 163 x VS

#     output_model_prob = []
#     for i,val in enumerate(resp_mask):
#         if val == 0: 
#             continue
#         else:
#             prob_distribution = next_token_prediction_values(logp, i-1).squeeze(0) # so to get prev one -> 1xVS 
#             correct_token_idx = input_ids[i] # correct / incorrect input idx ..  
#             val = prob_distribution[correct_token_idx]
#             print('Token should have been : ', tokenizer.decode([correct_token_idx]), ' The probability I am getting out from this is : ', val.item()) 
#             output_model_prob.append(val)

#     print(type(output_model_prob))
    
#     return torch.stack(output_model_prob).mean()


#### Training Loop


Prompt: 'How to pet my dog?'

Answer: 

token-1 : 'Petting'
token-2 : 'your'
token-3 : 'dog'
token-4 : 'is'
token-5 : 'a'
token-6 : 'good'
token-7 : 'relaxation'
... etc 



output 1 : 


In [ ]:

from trl import AutoModelForCausalLMWithValueHead
reward_model = AutoModelForCausalLMWithValueHead.from_pretrained(BASE_MODEL, cache_dir = FILE_PATH)

reward_model.to(DEVICE)

In [ ]:

vals = 'I am a good boy'
ds = tokenizer(vals).input_ids
ds = torch.tensor(data = ds).unsqueeze(0).to(DEVICE)
a = reward_model.forward(ds)
# print(a[1][-1])
a[2][:,-1]

# tokenizer.decode(*a)

In [ ]:
a[2], a[2][:,-1]

In [ ]:

from transformers import GenerationConfig
# https://huggingface.co/docs/transformers/en/main_classes/text_generation


In [ ]:

def sampling_outputs(model , data , n=5):
    generation_config = GenerationConfig.from_pretrained(BASE_MODEL, cache_dir=FILE_PATH)
    
    generation_config.max_new_length = 400
    generation_config.temperature= 0.6
    generation_config.use_cache = False
    generation_config.do_sample = True
    
    input_ids = torch.tensor(data['input_ids']).unsqueeze(0).to(DEVICE)

    outs = []
    for _ in range(n):
        # print('inside the loop !!')
        # print("The input to the model is " , tokenizer.decode(*input_ids) )
        out = model.generate(input_ids , generation_config = generation_config)
        generated_output = out[:, input_ids.shape[1]:]

        # print('model outputs is : ',  generated_output  , generated_output.device)
        # out = tokenizer.decode(*out) # this is giving the whole input + the output why ??
        out = tokenizer.decode(*generated_output) # this is giving the whole input + the output why ??
        
        outs.append(generated_output)

    return outs


def scoring_outputs(reward_model, outputs):
    
    # if type(outputs) == list:
    #     print("Type of outputs is : ", type(outputs), outputs)

    rewards = []
    for output in outputs: 
        tokenized_data = output
        input_ids = tokenized_data
        attention_mask = [1] * output.shape[-1]
        attention_mask = torch.tensor(data = attention_mask).to(DEVICE).unsqueeze(0)

        reward = reward_model.forward(input_ids , attention_mask = attention_mask)
        reward = reward[2][:,-1]
        rewards.append(reward)

    return rewards        
    

def group_computation(advantages) -> torch.Tensor:
    '''
    Group computation 
    '''
    advantages = torch.stack(advantages).float()
    return  (advantages - advantages.mean()) / advantages.std()


data = torch.tensor(data= 1)
data2 = torch.tensor(data= 2)
reward = [1,2,3,4,45]
reward = [data, data, data, data2]

print(group_computation(reward))



In [ ]:
def grpo_loss(prompt: torch.Tensor, outputs:torch.Tensor, advantages:torch.Tensor, policy_model , reference_model, old_model, beta:torch.Tensor):
    '''
    GRPO loss 
    so its requires the prompt , outputs and averages over the output values 

    loss goes like this :
    old_policy : keep the t-1 policy weights as we need to importance sampling ( # TODO : more on this )
    policy : the one for which we are optimizing 
    reference model : the reference / fixed model 
    '''
    import gc 
    gc.collect()
    torch.mps.empty_cache()


    # print('type of prompt is : ', type(prompt), prompt)
    # print('type of outputs is : ', type(outputs), outputs)
    # print('type of advantages is : ', type(advantages), advantages)
    # print('type of beta is : ', type(beta))
    
    if type(prompt) is list: 
        prompt = torch.tensor(prompt).to(DEVICE)

    # if type(beta) is float:
    beta = torch.tensor(beta).to(DEVICE)
    
    # print('type of prompt is : ', type(prompt), prompt)
    # print('beta shape is ', beta.shape)

    policy_model.train()
    reference_model.eval()
    old_model.eval()

    
    assert len(outputs) == len(advantages) , 'The no. of samples of the outputs generated should be equal to no. of advantages processed '

    loss = torch.tensor(data = 0.0).to(DEVICE)
    for idx, output in enumerate(outputs): 
        advantage = advantages[idx]
        output = output.squeeze(0).to(DEVICE)
        total_sum_of_probs = torch.tensor(data = 0.0).to(DEVICE)
        for t, output_token in enumerate(output): 
            # check the probability of getting this token , given input prompt + tokens till this
            # print('prompt is ', type(prompt) , prompt)
            # print('output tokens are : ', type(output[:t]), output[:t])
            input_tokens = torch.cat((prompt, output[:t])).unsqueeze(0)
            policy_model_prob = policy_model(input_tokens)


            policy_model_prob = policy_model_prob.logits[:,-1, output_token] # Output will be a tensor of Vocab_Size (VS)
            with torch.no_grad():
                reference_model_prob = reference_model(input_tokens).logits[:,-1, output_token]
                old_model_prob = old_model(input_tokens).logits[:,-1, output_token]

            assert policy_model_prob.shape == reference_model_prob.shape , 'The output shapes of the probabilities should be of same shape '

            first_term = ( policy_model_prob / old_model_prob ) * advantage  
            second_term = beta * (reference_model_prob / policy_model_prob)
            third_term = beta * (torch.log(reference_model_prob / policy_model_prob))
            fourth_term = beta

            ans = (first_term - second_term + third_term + fourth_term).squeeze()
            total_sum_of_probs += ans

            # Clean up explicitly if needed
            del input_tokens, policy_model_prob, reference_model_prob, old_model_prob
            torch.mps.empty_cache()
            gc.collect()


        loss += total_sum_of_probs / len(output)
    
    return loss / len(outputs)



In [ ]:
data = [[16096,    28,   338,   417,    99,   253,  1109,  2359,    30, 17579,
           253,  1165,  1902,   282,   314,   371, 11899,    92,  4657,   288]]

a = torch.tensor(data)
a[:, :2]

In [ ]:


input_ids = torch.tensor(dataset[0]['input_ids']).unsqueeze(0)
attention_mask = torch.tensor(dataset[0]['attention_mask']).unsqueeze(0)
print(input_ids)
outs =model.generate(input_ids)
print(outs.shape)

print(outs)
print(tokenizer.decode(*outs))



In [ ]:
dataset[0]

In [ ]:

import gc 
gc.collect()
torch.mps.empty_cache()

import copy
import torch.nn as nn
import torch.nn.functional as F


#model definition 
policy_model = model
reference_model = copy.deepcopy(model).eval()
reward_model = reward_model

for param in reference_model.parameters():
    param.requires_grad = False
    param.grad = None

optim = torch.optim.AdamW(policy_model.parameters(), lr = 5e-8)

# dataset = dataset.map(tokenizer_mapping)
device = DEVICE
policy_model = policy_model.to(device)
reference_model = reference_model.to(device) 
reward_model = reward_model.to(device)
# dataset = dataset.to(device)

print("The models are on : ", policy_model.device)

outputs = None

def training_loop(epochs = 1): 
    global outputs
    for epoch in range(0, epochs):
        for idx, data in enumerate(dataset):
            gc.collect()
            torch.mps.empty_cache()

            optim.zero_grad() # we dont like accumulation !! 
    
            outputs = sampling_outputs(policy_model, data, n=4)                    
            rewards = scoring_outputs(reward_model, outputs)

            assert len(outputs) == len(rewards) , "The shape mismatch should not occur"
            advantages = group_computation(rewards)
            
            # Get the final advantages here !! 

            # loss function implementation !! from : https://arxiv.org/pdf/2402.03300

            loss = grpo_loss(data['input_ids'] , outputs , advantages, policy_model , policy_model , policy_model ,  beta=0.4)
            print('LOSS : ' ,loss.item() , 'EPOCH : ', epoch)

            loss.backward() # cals done
            optim.step() # backprop

            break

training_loop(1)


https://arxiv.org/pdf/2402.03300